<a href="https://colab.research.google.com/github/yonabell/Report/blob/main/Reporting_final_python_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import re

In [2]:
from google.colab import files
uploaded = files.upload()

Saving 2023_sample_data1.xlsx to 2023_sample_data1 (4).xlsx


In [3]:
file_path_previous_year = '2023_sample_data1.xlsx'

In [ ]:
# file_path_current_year = 'path_to_input_folder/2024_sample_data.xlsx'
# file_path_previous_year = 'path_to_input_folder/2023_sample_data.xlsx'
# output_folder_path = 'path_to_output_folder/'

In [4]:
# Defining a function for the following data cleaning steps
def clean_data(file_path):
    # Load the data with no predefined header
    cpt_df = pd.read_excel(file_path, engine='openpyxl', header=4)

    # Filtering rows and extracting only 'A' followed by nine digits
    filtered_cpt_df = cpt_df[cpt_df['Unnamed: 0'].str.extract(r'(A\d{9})', expand=False).notna()].copy()

    # Extracting the ID
    filtered_cpt_df['ID'] = filtered_cpt_df['Unnamed: 0'].str.extract(r'(A\d{9})', expand=False)

    # Reorder columns to place 'ID' next to 'Unnamed: 0'
    cols = filtered_cpt_df.columns.tolist()
    id_index = cols.index('ID')
    cols = cols[:1] + [cols[id_index]] + cols[1:id_index] + cols[id_index + 1:]
    filtered_cpt_df = filtered_cpt_df[cols]

    # Drop 'Unnamed: 0' column
    filtered_cpt_df = filtered_cpt_df.drop('Unnamed: 0', axis=1)

    # Drop specific columns (1 to 7)
    filtered_cpt_df = filtered_cpt_df.drop(filtered_cpt_df.columns[1:8], axis=1)

    # Select every 7th column starting from the 8th column (index 7)
    # Unnecessary column for the analysis with no data
    column_names = filtered_cpt_df.columns.tolist()
    columns_to_drop = column_names[7::7]

    # Drop the selected columns from the DataFrame
    filtered_cpt_df.drop(columns=columns_to_drop, inplace=True)

    return filtered_cpt_df

In [5]:
# Call the clean_data function
filtered_cpt_df1 = clean_data(file_path_previous_year) # file_path_current_year

In [6]:
# Display the filtered_cpt_df
filtered_cpt_df1

,ID,Purch. Volume.1,S-Revenues.1,Profit Margin I.1,PM I % of Purchased Volume.1,Profit Margin I*.1,PM I* % of Purchased Volume.1,Purch. Volume.2,S-Revenues.2,Profit Margin I.2,...,Profit Margin I.11,PM I % of Purchased Volume.11,Profit Margin I*.11,PM I* % of Purchased Volume.11,Purch. Volume.12,S-Revenues.12,Profit Margin I.12,PM I % of Purchased Volume.12,Profit Margin I*.12,PM I* % of Purchased Volume.12
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,...,34.0,9.8384,45.0,2.033,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,...,643.0,15.7322,542.0,2.519,543.0,725.0,643.0,15.7322,542.0,2.519


In [7]:
filtered_cpt_df1.shape

(2, 73)

In [8]:
def replace_suffix_with_months(df):
    """
    This function replaces numerical suffixes in column names with corresponding month names
    and replaces dots with spaces.

    Args:
    df (pd.DataFrame): The input DataFrame whose column names need to be updated.

    Returns:
    pd.DataFrame: The DataFrame with updated column names.
    """
    # Mapping of number suffixes to month names
    month_map = {
        '.1': ' Jan',
        '.2': ' Feb',
        '.3': ' Mar',
        '.4': ' Apr',
        '.5': ' May',
        '.6': ' Jun',
        '.7': ' Jul',
        '.8': ' Aug',
        '.9': ' Sep',
        '.10': ' Oct',
        '.11': ' Nov',
        '.12': ' Dec'
    }

    # Replace dots with spaces and map suffixes at the end of column names to respective months
    new_column_names = []
    for column_name in df.columns:
        new_column_name = column_name.replace('.', ' ')  # Replace dots with spaces
        for suffix, month_name in month_map.items():
            # Ensure we replace only if the suffix appears at the end of the column name
            if new_column_name.endswith(suffix.replace('.', ' ')):
                new_column_name = new_column_name.replace(suffix.replace('.', ' '), month_name)
        new_column_names.append(new_column_name)

    # Assign the new column names to the dataframe
    df.columns = new_column_names

    return df

In [9]:
filtered_cpt_df1 = replace_suffix_with_months(filtered_cpt_df1)

In [10]:
filtered_cpt_df1

,ID,Purch Volume Jan,S-Revenues Jan,Profit Margin I Jan,PM I % of Purchased Volume Jan,Profit Margin I* Jan,PM I* % of Purchased Volume Jan,Purch Volume Feb,S-Revenues Feb,Profit Margin I Feb,...,Profit Margin I Nov,PM I % of Purchased Volume Nov,Profit Margin I* Nov,PM I* % of Purchased Volume Nov,Purch Volume Dec,S-Revenues Dec,Profit Margin I Dec,PM I % of Purchased Volume Dec,Profit Margin I* Dec,PM I* % of Purchased Volume Dec
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,...,34.0,9.8384,45.0,2.033,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,...,643.0,15.7322,542.0,2.519,543.0,725.0,643.0,15.7322,542.0,2.519


In [11]:
print(filtered_cpt_df1.shape)

(2, 73)


In [12]:
def rename_columns(filtered_cpt_df):
    # Define the new column name mappings for replacements at the start of column names
    new_column_names = {
        'ID': 'BPID',
        'Purch Volume': 'PurchVolume',
        'S-Revenues': 'Revenue',
        'Profit Margin I*': 'PMIStar',  # Handle the specific case for PMIStar first
        'Profit Margin I': 'PMI',
        'PM I % of Purchased Volume': 'PMIRate',
        'PM I* % of Purchased Volume': 'PMIStarRate'
    }

    # Iterate through the columns and apply the renaming based on the starting part of the name
    updated_columns = []
    for column in filtered_cpt_df.columns:
        # Check if the column starts with any of the keys in new_column_names
        for old_name, new_name in new_column_names.items():
            if column.startswith(old_name):
                # Replace the old part of the column name with the new one and keep the suffix (e.g., "Jan", "Feb")
                new_column = column.replace(old_name, new_name, 1)
                updated_columns.append(new_column)
                break
        else:
            # If no match, keep the column name unchanged
            updated_columns.append(column)

    # Rename the columns in the dataframe
    filtered_cpt_df.columns = updated_columns

    return filtered_cpt_df


In [13]:
filtered_cpt_df1 = rename_columns(filtered_cpt_df1)

In [14]:
filtered_cpt_df1

,BPID,Purch Volume Jan,Revenue Jan,PMI Jan,PMIRate Jan,PMIStar Jan,PMIStarRate Jan,Purch Volume Feb,Revenue Feb,PMI Feb,...,PMI Nov,PMIRate Nov,PMIStar Nov,PMIStarRate Nov,Purch Volume Dec,Revenue Dec,PMI Dec,PMIRate Dec,PMIStar Dec,PMIStarRate Dec
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,...,34.0,9.8384,45.0,2.033,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,...,643.0,15.7322,542.0,2.519,543.0,725.0,643.0,15.7322,542.0,2.519


In [15]:
filtered_cpt_df1.columns

Index(['BPID', 'Purch  Volume Jan', 'Revenue Jan', 'PMI Jan', 'PMIRate Jan',
       'PMIStar Jan', 'PMIStarRate Jan', 'Purch  Volume Feb', 'Revenue Feb',
       'PMI Feb', 'PMIRate Feb', 'PMIStar Feb', 'PMIStarRate Feb',
       'Purch  Volume Mar', 'Revenue Mar', 'PMI Mar', 'PMIRate Mar',
       'PMIStar Mar', 'PMIStarRate Mar', 'Purch  Volume Apr', 'Revenue Apr',
       'PMI Apr', 'PMIRate Apr', 'PMIStar Apr', 'PMIStarRate Apr',
       'Purch  Volume May', 'Revenue May', 'PMI May', 'PMIRate May',
       'PMIStar May', 'PMIStarRate May', 'Purch  Volume Jun', 'Revenue Jun',
       'PMI Jun', 'PMIRate Jun', 'PMIStar Jun', 'PMIStarRate Jun',
       'Purch  Volume Jul', 'Revenue Jul', 'PMI Jul', 'PMIRate Jul',
       'PMIStar Jul', 'PMIStarRate Jul', 'Purch  Volume Aug', 'Revenue Aug',
       'PMI Aug', 'PMIRate Aug', 'PMIStar Aug', 'PMIStarRate Aug',
       'Purch  Volume Sep', 'Revenue Sep', 'PMI Sep', 'PMIRate Sep',
       'PMIStar Sep', 'PMIStarRate Sep', 'Purch  Volume Oct', 'Revenue 

In [16]:
from google.colab import files
uploaded = files.upload()

Saving 2024_sample_data.xlsx to 2024_sample_data (3).xlsx


In [17]:
file_path_current_year = '2024_sample_data.xlsx'

In [18]:
# Call the clean_data function
filtered_cpt_df2 = clean_data(file_path_current_year)

In [19]:
filtered_cpt_df2

,ID,Purch. Volume.1,S-Revenues.1,Profit Margin I.1,PM I % of Purchased Volume.1,Profit Margin I*.1,PM I* % of Purchased Volume.1,Purch. Volume.2,S-Revenues.2,Profit Margin I.2,PM I % of Purchased Volume.2,Profit Margin I*.2,PM I* % of Purchased Volume.2,Purch. Volume.3,S-Revenues.3,Profit Margin I.3,PM I % of Purchased Volume.3,Profit Margin I*.3,PM I* % of Purchased Volume.3
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [20]:
# Assuming 'filtered_cpt_df2' is your DataFrame
filtered_cpt_df2 = replace_suffix_with_months(filtered_cpt_df2)

In [21]:
filtered_cpt_df2

,ID,Purch Volume Jan,S-Revenues Jan,Profit Margin I Jan,PM I % of Purchased Volume Jan,Profit Margin I* Jan,PM I* % of Purchased Volume Jan,Purch Volume Feb,S-Revenues Feb,Profit Margin I Feb,PM I % of Purchased Volume Feb,Profit Margin I* Feb,PM I* % of Purchased Volume Feb,Purch Volume Mar,S-Revenues Mar,Profit Margin I Mar,PM I % of Purchased Volume Mar,Profit Margin I* Mar,PM I* % of Purchased Volume Mar
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [22]:
filtered_cpt_df2 = rename_columns(filtered_cpt_df2)

In [23]:
filtered_cpt_df2

,BPID,Purch Volume Jan,Revenue Jan,PMI Jan,PMIRate Jan,PMIStar Jan,PMIStarRate Jan,Purch Volume Feb,Revenue Feb,PMI Feb,PMIRate Feb,PMIStar Feb,PMIStarRate Feb,Purch Volume Mar,Revenue Mar,PMI Mar,PMIRate Mar,PMIStar Mar,PMIStarRate Mar
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [24]:
# Assuming df1 is filtered_cpt_df1 and you want to append columns from filtered_cpt_df2 starting from the second column

combined_df = pd.concat([filtered_cpt_df1, filtered_cpt_df2.iloc[:, 1:]], axis=1)


In [25]:
combined_df

,BPID,Purch Volume Jan,Revenue Jan,PMI Jan,PMIRate Jan,PMIStar Jan,PMIStarRate Jan,Purch Volume Feb,Revenue Feb,PMI Feb,...,PMI Feb,PMIRate Feb,PMIStar Feb,PMIStarRate Feb,Purch Volume Mar,Revenue Mar,PMI Mar,PMIRate Mar,PMIStar Mar,PMIStarRate Mar
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,...,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,...,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [26]:
filtered_cpt_df1.columns

Index(['BPID', 'Purch  Volume Jan', 'Revenue Jan', 'PMI Jan', 'PMIRate Jan',
       'PMIStar Jan', 'PMIStarRate Jan', 'Purch  Volume Feb', 'Revenue Feb',
       'PMI Feb', 'PMIRate Feb', 'PMIStar Feb', 'PMIStarRate Feb',
       'Purch  Volume Mar', 'Revenue Mar', 'PMI Mar', 'PMIRate Mar',
       'PMIStar Mar', 'PMIStarRate Mar', 'Purch  Volume Apr', 'Revenue Apr',
       'PMI Apr', 'PMIRate Apr', 'PMIStar Apr', 'PMIStarRate Apr',
       'Purch  Volume May', 'Revenue May', 'PMI May', 'PMIRate May',
       'PMIStar May', 'PMIStarRate May', 'Purch  Volume Jun', 'Revenue Jun',
       'PMI Jun', 'PMIRate Jun', 'PMIStar Jun', 'PMIStarRate Jun',
       'Purch  Volume Jul', 'Revenue Jul', 'PMI Jul', 'PMIRate Jul',
       'PMIStar Jul', 'PMIStarRate Jul', 'Purch  Volume Aug', 'Revenue Aug',
       'PMI Aug', 'PMIRate Aug', 'PMIStar Aug', 'PMIStarRate Aug',
       'Purch  Volume Sep', 'Revenue Sep', 'PMI Sep', 'PMIRate Sep',
       'PMIStar Sep', 'PMIStarRate Sep', 'Purch  Volume Oct', 'Revenue 

In [27]:
filtered_cpt_df2.columns

Index(['BPID', 'Purch  Volume Jan', 'Revenue Jan', 'PMI Jan', 'PMIRate Jan',
       'PMIStar Jan', 'PMIStarRate Jan', 'Purch  Volume Feb', 'Revenue Feb',
       'PMI Feb', 'PMIRate Feb', 'PMIStar Feb', 'PMIStarRate Feb',
       'Purch  Volume Mar', 'Revenue Mar', 'PMI Mar', 'PMIRate Mar',
       'PMIStar Mar', 'PMIStarRate Mar'],
      dtype='object')

In [28]:
combined_df.columns

Index(['BPID', 'Purch  Volume Jan', 'Revenue Jan', 'PMI Jan', 'PMIRate Jan',
       'PMIStar Jan', 'PMIStarRate Jan', 'Purch  Volume Feb', 'Revenue Feb',
       'PMI Feb', 'PMIRate Feb', 'PMIStar Feb', 'PMIStarRate Feb',
       'Purch  Volume Mar', 'Revenue Mar', 'PMI Mar', 'PMIRate Mar',
       'PMIStar Mar', 'PMIStarRate Mar', 'Purch  Volume Apr', 'Revenue Apr',
       'PMI Apr', 'PMIRate Apr', 'PMIStar Apr', 'PMIStarRate Apr',
       'Purch  Volume May', 'Revenue May', 'PMI May', 'PMIRate May',
       'PMIStar May', 'PMIStarRate May', 'Purch  Volume Jun', 'Revenue Jun',
       'PMI Jun', 'PMIRate Jun', 'PMIStar Jun', 'PMIStarRate Jun',
       'Purch  Volume Jul', 'Revenue Jul', 'PMI Jul', 'PMIRate Jul',
       'PMIStar Jul', 'PMIStarRate Jul', 'Purch  Volume Aug', 'Revenue Aug',
       'PMI Aug', 'PMIRate Aug', 'PMIStar Aug', 'PMIStarRate Aug',
       'Purch  Volume Sep', 'Revenue Sep', 'PMI Sep', 'PMIRate Sep',
       'PMIStar Sep', 'PMIStarRate Sep', 'Purch  Volume Oct', 'Revenue 

In [29]:
# Assign month-year pairs without changing column names
def assign_month_year_mapping(combined_df, start_year=2023):
    # Initialize the mapping dictionary
    month_year_map = {}

    # Initialize month and year counters
    month = 1
    year = start_year

    # Loop through columns starting from the second column (since first is 'BPID')
    for i, column in enumerate(combined_df.columns[1:], start=1):  # Skip 'BPID'
        # Assign the current column a month-year label
        month_year_map[column] = f'{month}.{year}'

        # Increment the month
        month += 1

        # If month exceeds 12, reset to January and increment the year
        if month > 12:
            month = 1
            year += 1

    return month_year_map

# Example usage
month_year_map = assign_month_year_mapping(combined_df)
print(month_year_map)  # Just to verify the mapping


{'Purch  Volume Jan': '1.2029', 'Revenue Jan': '2.2029', 'PMI Jan': '3.2029', 'PMIRate Jan': '4.2029', 'PMIStar Jan': '5.2029', 'PMIStarRate Jan': '6.2029', 'Purch  Volume Feb': '7.2029', 'Revenue Feb': '8.2029', 'PMI Feb': '9.2029', 'PMIRate Feb': '10.2029', 'PMIStar Feb': '11.2029', 'PMIStarRate Feb': '12.2029', 'Purch  Volume Mar': '1.2030', 'Revenue Mar': '2.2030', 'PMI Mar': '3.2030', 'PMIRate Mar': '4.2030', 'PMIStar Mar': '5.2030', 'PMIStarRate Mar': '6.2030', 'Purch  Volume Apr': '7.2024', 'Revenue Apr': '8.2024', 'PMI Apr': '9.2024', 'PMIRate Apr': '10.2024', 'PMIStar Apr': '11.2024', 'PMIStarRate Apr': '12.2024', 'Purch  Volume May': '1.2025', 'Revenue May': '2.2025', 'PMI May': '3.2025', 'PMIRate May': '4.2025', 'PMIStar May': '5.2025', 'PMIStarRate May': '6.2025', 'Purch  Volume Jun': '7.2025', 'Revenue Jun': '8.2025', 'PMI Jun': '9.2025', 'PMIRate Jun': '10.2025', 'PMIStar Jun': '11.2025', 'PMIStarRate Jun': '12.2025', 'Purch  Volume Jul': '1.2026', 'Revenue Jul': '2.2026'

In [30]:
def filter_by_date_range(combined_df, month_year_map, start_month, end_month):
    # Find the columns that correspond to the start and end months
    columns_in_range = []

    # Flag to start capturing columns within the range
    in_range = False

    for column in combined_df.columns[1:]:  # Exclude BPID (first column)
        # Get the assigned month-year from the map
        if column in month_year_map:
            month_year = month_year_map[column]

            # Check if we hit the start month or are already in the range
            if month_year == start_month:
                in_range = True

            # Add columns if we're within the range
            if in_range:
                columns_in_range.append(column)

            # Stop once we pass the end month
            if month_year == end_month:
                break

    # Return the filtered data
    return combined_df[['BPID'] + columns_in_range]




In [29]:
# Example usage
start_month = '11.2023'
end_month = '1.2024'
filtered_data = filter_by_date_range(combined_df, month_year_map, start_month, end_month)
print(filtered_data)

In [31]:
filtered_data

,BPID
5,A987654321
8,A123456789


In [34]:
filtered_df.columns

Index(['BPID', 'Purch 11.2023', 'Revenue 11.2023', 'PMI 11.2023',
       'PMIRate 11.2023', 'PMIStar 11.2023', 'PMIStarRate 11.2023',
       'Purch 12.2023', 'Revenue 12.2023', 'PMI 12.2023', 'PMIRate 12.2023',
       'PMIStar 12.2023', 'PMIStarRate 12.2023', 'Purch 1.2024',
       'Revenue 1.2024', 'PMI 1.2024', 'PMIRate 1.2024', 'PMIStar 1.2024',
       'PMIStarRate 1.2024'],
      dtype='object')

In [ ]:
# Saving current year aggregated data to output folder
# aggregated_df1.to_excel(output_folder_path + '2024_aggregated.xlsx', index=False)

In [ ]:
# Saving current year aggregated data to output folder
# aggregated_df1.to_excel(output_folder_path + '2023_aggregated.xlsx', index=False)

In [ ]:
# Saving the final combined sum to output folder
# aggregated_df_sum.to_excel(output_folder_path + 'cpt_annual_profitability.xlsx', index=False)

In [ ]:
# Get the current and previous year dynamically
current_year = datetime.now().year
previous_year = current_year - 1

# Define input folder and output folder paths
input_folder_path = 'path_to_your_input/'
output_folder_path = 'path_to_output_folder/'

# Assign file paths dynamically
current_year_file_path = os.path.join(input_folder_path, f'{current_year}_sample_data.xlsx')
previous_year_file_path = os.path.join(input_folder_path, f'{previous_year}_sample_data.xlsx')

# Saving aggregated data for the current year
aggregated_df1.to_excel(os.path.join(output_folder_path, f'{current_year}_aggregated.xlsx'), index=False)

# Saving aggregated data for the previous year
aggregated_df2.to_excel(os.path.join(output_folder_path, f'{previous_year}_aggregated.xlsx'), index=False)
